# Install python packages with version to avoid conflicts

In [ ]:
# Run it twice if you have a warning
%pip install plotly==5.14.1 mercury==2.3.0 jupyter-dash==0.4.2 ipywidgets==8.0.3

# Imports

In [ ]:
from surianalytics.connectors import ESQueryBuilder, escape as es_escape
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import plotly.express as px
import mercury as mr
import ipywidgets as widgets
import json
from IPython.display import display
from django.utils import timezone
from datetime import timedelta, timezone as dt_tz
from django.conf import settings

# Some conf

In [ ]:
try:
    # allow to use timezone
    settings.configure()
    settings.USE_TZ = True
except:
    pass

# disable insecure warning
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# Be careful with max_rows, too much rows leads to freeze browser/os
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Build ES query example

## Builder

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-tls-*')
builder.set_page_size(0)
builder.set_tenant(1)

## Filters

In [ ]:
filters = [f'proto: {es_escape("UDP")}', f'proto: {es_escape("TCP")}']
filter2 = 'event_type: tls'
qfilter = ESQueryBuilder.filter_join(filters, operator='OR')
qfilter = ESQueryBuilder.filter_join([filter2, qfilter], operator='AND')
builder.set_qfilter(qfilter)

## Aggreagations

In [ ]:
builder.add_aggs('tls.sni.keyword', order='_count', sort='asc', size=5)
builder.add_aggs('tls.cipher_security.keyword', order='_count', sort='asc', size=5)
builder.add_aggs('tls.toto', order='_count', sort='desc', size=5)

## What does look like query

In [ ]:
JSON(builder.__dict__(), expanded=True)

In [ ]:
print(builder.tenant, builder.aggs, builder.qfilter, builder.index)

## Reset filters/aggs/index/tenants

In [ ]:
builder.reset()
JSON(builder.__dict__(), expanded=True)

In [ ]:
print(builder.tenant, builder.aggs, builder.qfilter, builder.index)

# Old TLS versions

## Cipher suite count on degraded / insecure

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-tls-*')
builder.set_page_size(0)
builder.set_tenant(0)

In [ ]:
filters = [f'tls.cipher_security: {es_escape("insecure")}', f'tls.cipher_security: {es_escape("degraded")}']
filter2 = 'event_type: tls'
qfilter = ESQueryBuilder.filter_join(filters, operator='OR')
qfilter = ESQueryBuilder.filter_join([filter2, qfilter], operator='AND')
builder.set_qfilter(qfilter)

builder.add_aggs('tls.cipher_suite.keyword', order='_count', sort='desc', size=5)

builder.set_from_date('2023-06-19T15:15:49+00:00')
builder.set_to_date('2023-06-19T20:42:49+00:00')

r = builder.post()
r

In [ ]:
content = r.json()

res = {'ciphers': [], 'count': []}
for key, val in content.get('aggregations', {}).items():
    for item in val.get('buckets', []):
        res['ciphers'].append(item['key'])
        res['count'].append(item['doc_count'])
        
df = pd.DataFrame({'Cipher Suite': res['ciphers'], 'Count': res['count']})
plot = df.groupby(['Cipher Suite']).sum().plot(kind='pie', y='Count', autopct='%1.0f%%')
df

## Global number of assets using insecure and degraded ciphers

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-tls-*')
builder.set_page_size(0)
builder.set_tenant(0)

In [ ]:
filters = [f'tls.cipher_security: {es_escape("insecure")}', f'tls.cipher_security: {es_escape("degraded")}']
filter2 = 'event_type: tls'
qfilter = ESQueryBuilder.filter_join(filters, operator='OR')
qfilter = ESQueryBuilder.filter_join([filter2, qfilter], operator='AND')
builder.set_qfilter(qfilter)

builder.add_aggs('tls.cipher_security.keyword', order='_count', sort='desc', size=5)

builder.set_from_date('2023-06-19T15:15:49+00:00')
builder.set_to_date('2023-06-19T20:42:49+00:00')

r = builder.post()
r

In [ ]:
content = r.json()

res = {'ciphers': [], 'count': []}
for key, val in content.get('aggregations', {}).items():
    for item in val.get('buckets', []):
        res['ciphers'].append(item['key'])
        res['count'].append(item['doc_count'])
        
df = pd.DataFrame({'Cipher Suite': res['ciphers'], 'Count': res['count']})
df

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-tls-*')
builder.set_page_size(0)
builder.set_tenant(0)

filters = [f'tls.cipher_security: {es_escape("insecure")}', f'tls.cipher_security: {es_escape("degraded")}']
filter2 = 'event_type: tls'
qfilter = ESQueryBuilder.filter_join(filters, operator='OR')
qfilter = ESQueryBuilder.filter_join([filter2, qfilter], operator='AND')
builder.set_qfilter(qfilter)

builder.add_aggs('tls.cipher_security.keyword', order='_count', sort='desc', size=5)
builder.add_aggs('tls.cipher_suite.keyword', order='_count', sort='desc', size=5)

builder.set_from_date('2023-06-14T09:42:49+00:00')
builder.set_to_date('2023-06-14T14:42:49+00:00')

r = builder.post()
r

In [ ]:
content = r.json()

sunburst = []
tree = []
for key, val in content.get('aggregations', {}).items():
    for item in val.get('buckets', []):
        for idx, sub_item in enumerate(item['2'].get('buckets', [])):
            sunburst.append((item['key'], item['doc_count'], sub_item['key'], sub_item['doc_count']))
            if idx == 0:
                tree.append((item['key'], item['doc_count'], sub_item['key'], sub_item['doc_count']))
            else:
                tree.append(('', '', sub_item['key'], sub_item['doc_count']))

df = pd.DataFrame(tree, columns =['Cipher Security', 'Security Count', 'Cipher Suite', 'Suite Count'])
df

In [ ]:
df = pd.DataFrame(sunburst, columns =['Cipher Security', 'Security Count', 'Cipher Suite', 'Suite Count'])
px.sunburst(df, path=['Cipher Security', 'Security Count', 'Cipher Suite', 'Suite Count'], width=800, height=800)

# Filter out host insight requests (scirius#6112)

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-host_id')
builder.set_page_size(50)
builder.set_tenant(1)
builder.set_time_filter('host_id.services.values.first_seen')
builder.set_from_date('2023-06-19T15:15:49+00:00')
builder.set_to_date('2023-06-19T20:42:49+00:00')

date_filter = '2023-06-14T13:53:27.941628+00:00'
app_proto_filter = 'dhcp'

filters = [f'host_id.services.values.first_seen: "{date_filter}"', f'host_id.services.values.app_proto.keyword: {app_proto_filter}']
qfilter = ESQueryBuilder.filter_join(filters, operator='AND')
qfilter = ESQueryBuilder.filter_join([qfilter, 'host_id.tenant: 1'], operator='AND')
builder.set_qfilter(qfilter)

JSON(builder.__dict__(), expanded=True)

In [ ]:
r = builder.post()
r

In [ ]:
content = r.json()

filters = {
    'values.first_seen': date_filter,
    'values.app_proto': app_proto_filter
}

dataframe = []
for hit in content.get('hits', {}).get('hits', []):
    services = hit.get('_source', {}).get('host_id', {}).get('services', [])
    services = ESQueryBuilder.clean_host_id(services, **filters)
    
    # build data frame
    for service in services:
        for idx, val in enumerate(service['values']):
            if idx == 0:
                dataframe.append((service['proto'], service['port'], val['first_seen'], val['last_seen'], val['app_proto']))
            else:
                dataframe.append(('', '', val['first_seen'], val['last_seen'], val['app_proto']))

df = pd.DataFrame(dataframe, columns =['Proto', 'Port', 'First Seen', 'Last Seen', 'App Proto'])
df

In [ ]:
builder = ESQueryBuilder()
builder.set_index('logstash-host_id')
builder.set_page_size(50)
# builder.set_tenant(1)
builder.set_time_filter('host_id.services.values.first_seen')

builder.set_from_date('2023-06-19T09:42:49+00:00')
builder.set_to_date('2023-06-19T14:42:49+00:00')

qfilter = ESQueryBuilder.filter_join(['host_id.services.proto: udp', 'ip: 10.7.5.5'], operator='AND')
builder.set_qfilter(qfilter)

r = builder.post()
r

In [ ]:
content = r.json()

filters = {'proto': 'udp'}

dataframe = []
for hit in content.get('hits', {}).get('hits', []):
    services = hit.get('_source', {}).get('host_id', {}).get('services', [])
    
    res = ESQueryBuilder.clean_host_id(services, **filters)
    services = res
    
    # build data frame
    for service in services:
        for val in service['values']:
            dataframe.append((hit['_source']['ip'], service['proto'], service['port'], val['first_seen'], val['last_seen'], val['app_proto']))

df = pd.DataFrame(dataframe, columns =['IP', 'Proto', 'Port', 'First Seen', 'Last Seen', 'App Proto'])
df.sort_values('IP')

In [ ]:
builder = ESQueryBuilder()
# builder.set_index('host_id-1')
builder.set_index('logstash-host_id')
builder.set_page_size(50)
# builder.set_tenant(1)
builder.set_time_filter('host_id.services.values.first_seen')
builder.set_from_date('2023-06-19T09:42:49+00:00')
builder.set_to_date('2023-06-19T14:42:49+00:00')

filter_proto = 'host_id.services.proto: udp'
filter_ja3 = 'host_id.tls.ja3.hash: 6734f37431670b3ab4292b8f60f29984'
filter_first_seen = 'host_id.tls.ja3.first_seen: "2023-06-19T17:42:12.018942+0200"'

#qfilter = ESQueryBuilder.filter_join([filter_proto, filter_ja3, filter_first_seen, 'host_id.tenant: 1'], operator='AND')
qfilter = ESQueryBuilder.filter_join([filter_proto, filter_ja3, filter_first_seen], operator='AND')
builder.set_qfilter(qfilter)

r = builder.post()
r.content

In [ ]:
content = r.json()

filter_proto = {'proto': 'udp', 'port': 53}
filter_ja3 = {'hash': '6734f37431670b3ab4292b8f60f29984', 'first_seen': '2023-06-19T17:42:12.018942+0200'}

dataframe_proto = []
dataframe_ja3 = []
for hit in content.get('hits', {}).get('hits', []):

    services = hit.get('_source', {}).get('host_id', {}).get('services', [])
    tls_ja3 = hit.get('_source', {}).get('host_id', {}).get('tls.ja3', [])

    services = ESQueryBuilder.clean_host_id(services, **filter_proto)
    tls_ja3 = ESQueryBuilder.clean_host_id(tls_ja3, **filter_ja3)

    # build data frame
    for service in services:
        for val in service['values']:
            dataframe_proto.append((hit['_source']['ip'], service['proto'], service['port'], val['first_seen'], val['last_seen'], val['app_proto']))
            
    for ja3 in tls_ja3:
        dataframe_ja3.append((hit['_source']['ip'], ja3['agent'], ja3['hash'], ja3['first_seen'], ja3['last_seen']))

In [ ]:
df = pd.DataFrame(dataframe_proto, columns =['IP', 'Proto', 'Port', 'First Seen', 'Last Seen', 'App Proto'])
df.sort_values('IP')

In [ ]:
df = pd.DataFrame(dataframe_ja3, columns =['IP', 'Agent', 'Hash', 'First Seen', 'Last Seen'])
df.sort_values('IP')

# widgets

In [ ]:
title = widgets.Label(value='Select time range')
from_date = widgets.DatetimePicker(description='Start date')
to_date = widgets.DatetimePicker(description='End date')

text_from_date = widgets.Text()
text_to_date = widgets.Text()

display(title, from_date, text_from_date, to_date, text_to_date)

# setting a and b avoid a useless output
a = widgets.jslink((from_date, 'value'), (text_from_date, 'value'))
b = widgets.jslink((to_date, 'value'), (text_to_date, 'value'))

In [ ]:
print(from_date)

builder = ESQueryBuilder()
builder.set_index('logstash-host_id')
builder.set_page_size(50)
builder.set_tenant(1)
builder.set_time_filter('host_id.services.values.first_seen')
builder.set_from_date(from_date.value.isoformat())
builder.set_to_date(to_date.value.isoformat())

JSON(builder.__dict__(), expanded=True)

builder2 = ESQueryBuilder()
builder2.set_index('host_id-1')
builder2.set_page_size(50)
builder2.set_tenant(1)
builder2.set_time_filter('host_id.services.values.first_seen')
builder2.set_from_date('2023-06-19T15:00:00+00:00')
builder2.set_to_date('2023-06-19T21:00:00+00:00')

d = builder.__dict__()
d

d2 = builder2.__dict__()
d2

display(d == d2)

In [ ]:
from_date_ts = int((timezone.now() - timedelta(weeks=365)).timestamp())
to_date_ts = int(timezone.now().timestamp())

print(from_date_ts)

title = widgets.Label(value='Select time range')
from_date = widgets.IntSlider(description='Start date', min=from_date_ts, max=to_date_ts, continuous_update=True)
to_date = widgets.IntSlider(description='End date', min=from_date_ts, max=to_date_ts, continuous_update=True)

output = widgets.Label()
def update(change):
    with output:
        print(f'{change["owner"].description}: {datetime.utcfromtimestamp(change["new"]).astimezone(tz=dt_tz.utc).isoformat()}')

# a = widgets.jslink((from_date, 'value'), (output, 'value'))
# a = widgets.jslink((to_date, 'value'), (output, 'value'))

from_date.observe(update, names="value")
to_date.observe(update, names="value")

display(title, from_date, to_date, output)

In [ ]:
builder.get_data?